<font size="+3"><b>Assignment 4: Pipelines and Hyperparameter Tuning</b></font>

***
* **Full Name** = Emiko Emiko
* **UCID** = 30161505
***

<font color='Blue'>
In this assignment, you will be putting together everything you have learned so far. You will need to find your own dataset, do all the appropriate preprocessing, test different supervised learning models, and evaluate the results. More details for each step can be found below. You will also be asked to describe the process by which you came up with the code. More details can be found below. Please cite any websites or AI tools that you used to help you with this assignment.
</font>

<font color='Red'>
For this assignment, in addition to your .ipynb file, please also attach a PDF file. To generate this PDF file, you can use the print function (located under the "File" within Jupyter Notebook). Name this file ENGG444_Assignment##__yourUCID.pdf (this name is similar to your main .ipynb file). We will evaluate your assignment based on the two files and you need to provide both.
</font>


|         **Question**         | **Point(s)** |
|:----------------------------:|:------------:|
|  **1. Preprocessing Tasks**  |              |
|              1.1             |       2      |
|              1.2             |       2      |
|              1.3             |       4      |
| **2. Pipeline and Modeling** |              |
|              2.1             |       3      |
|              2.2             |       6      |
|              2.3             |       5      |
|              2.4             |       3      |
|     **3. Bonus Question**    |     **2**    |
|           **Total**          |    **25**    |

## **0. Dataset**

This data is a subset of the **Heart Disease Dataset**, which contains information about patients with possible coronary artery disease. The data has **14 attributes** and **294 instances**. The attributes include demographic, clinical, and laboratory features, such as age, sex, chest pain type, blood pressure, cholesterol, and electrocardiogram results. The last attribute is the **diagnosis of heart disease**, which is a categorical variable with values from 0 (no presence) to 4 (high presence). The data can be used for **classification** tasks, such as predicting the presence or absence of heart disease based on the other attributes.

In [1]:
import pandas as pd

# Define the data source link
_link = 'https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.hungarian.data'

# Read the CSV file into a Pandas DataFrame, considering '?' as missing values
df = pd.read_csv(_link, na_values='?',
                 names=['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs',
                        'restecg', 'thalach', 'exang', 'oldpeak', 'slope',
                        'ca', 'thal', 'num'])

# Display the DataFrame
display(df)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,28,1,2,130.0,132.0,0.0,2.0,185.0,0.0,0.0,NaN,NaN,NaN,0
1,29,1,2,120.0,243.0,0.0,0.0,160.0,0.0,0.0,NaN,NaN,NaN,0
2,29,1,2,140.0,NaN,0.0,0.0,170.0,0.0,0.0,NaN,NaN,NaN,0
3,30,0,1,170.0,237.0,0.0,1.0,170.0,0.0,0.0,NaN,NaN,6.0,0
4,31,0,2,100.0,219.0,0.0,1.0,150.0,0.0,0.0,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,52,1,4,160.0,331.0,0.0,0.0,94.0,1.0,2.5,NaN,NaN,NaN,1
290,54,0,3,130.0,294.0,0.0,1.0,100.0,1.0,0.0,2.0,NaN,NaN,1
291,56,1,4,155.0,342.0,1.0,0.0,150.0,1.0,3.0,2.0,NaN,NaN,1
292,58,0,2,180.0,393.0,0.0,0.0,110.0,1.0,1.0,2.0,NaN,7.0,1


# **1. Preprocessing Tasks**

- **1.1** Find out which columns have more than 60% of their values missing and drop them from the data frame. Explain why this is a reasonable way to handle these columns. **(2 Points)**

- **1.2** For the remaining columns that have some missing values, choose an appropriate imputation method to fill them in. You can use the `SimpleImputer` class from `sklearn.impute` or any other method you prefer. Explain why you chose this method and how it affects the data. **(2 Points)**

- **1.3** Assign the `num` column to the variable `y` and the rest of the columns to the variable `X`. The `num` column indicates the presence or absence of heart disease based on the angiographic disease status of the patients. Create a `ColumnTransformer` object that applies different preprocessing steps to different subsets of features. Use `StandardScaler` for the numerical features, `OneHotEncoder` for the categorical features, and `passthrough` for the binary features. List the names of the features that belong to each group and explain why they need different transformations. You will use this `ColumnTransformer` in a pipeline in the next question. **(4 Points)**

<font color='Green'><b>Answer:</b></font>

- **1.1** We need to train our data to the point that it almosts overfits for unseen data, considering the fact that this model would be used to predict sensitive information regarding the possibility of a heart disease. Training the model with more than 60% missing data would more certainly than not cause bias in the model and underfitting for unseen data. The columns 'chol','fbs','slope','ca' and 'thal' all have more than 60% data missing and while we could find ways to fill in the missing values, the columns are mostly categorical. So in this case making the wrong assumptions could result in bad performance.

In [2]:
# 1.1
# Add necessary code here.
print(df.isna().sum() / len(df) * 100)#Print the missing value percentages here (code gotten from lab 6)

age          0.000000
sex          0.000000
cp           0.000000
trestbps     0.340136
chol         7.823129
fbs          2.721088
restecg      0.340136
thalach      0.340136
exang        0.340136
oldpeak      0.000000
slope       64.625850
ca          98.979592
thal        90.476190
num          0.000000
dtype: float64


In [4]:
#extract the columns with less than 60% missing values.

df = df.loc[:, ['age','sex','cp','trestbps','restecg','thalach','exang','oldpeak','num']]

print(df.isna().sum() / len(df) * 100)#print values of new dataset

age         0.000000
sex         0.000000
cp          0.000000
trestbps    0.340136
restecg     0.340136
thalach     0.340136
exang       0.340136
oldpeak     0.000000
num         0.000000
dtype: float64


<font color='Green'><b>Answer:</b></font>

- **1.2** .....................

In [5]:
# 1.2
# Add necessary code here.
#trestbps(resting blood pressure) is continuous integer value, so we can use mean
#restecg is categorical, use backfill or forward fill
#thalach(maximum heart rate achieved) is also continuous integer so we can use mean
#exang(exercise induced angina) is categorical, we'll use ffill
from sklearn.impute import SimpleImputer


#Use SimpleImputer for thalach and trestbps
imputer = SimpleImputer(strategy='mean')
columns_to_impute = ['trestbps', 'thalach']
df[columns_to_impute] = imputer.fit_transform(df[columns_to_impute])


#Use ffill for exang and restecg
df['restecg'] = df['restecg'].ffill()
df['exang'] = df['exang'].bfill()

#check to see if missing values are gone
print(df.isnull().sum())




age         0
sex         0
cp          0
trestbps    0
restecg     0
thalach     0
exang       0
oldpeak     0
num         0
dtype: int64


<font color='Green'><b>Answer:</b></font>

- **1.3** .....................
The columns with numerical values are age, trestbps, thalach, and oldpeak. The ones with categorical values are cp and restecg. The binary columns are sex and exang. Standard scaler is a technique for transforming numerical data to have a mean of zero and a standard deviation of one. It is useful for machine learning algorithms such as this one
that would perform better when the input variables are scaled to a standard range. One hot encoding is a technique to convert categorical variables into
numerical values for machine learning models.
It creates a new column for each category and assigns a binary value of 1 or 0 simply to indicate the presence or absence of that category. This allows for improved implementation of the ML model. The 'sex' and 'exang' columns are already binary, so we 'passthrough' to indicate that no changes should be made.



In [15]:
# 1.3
# Add necessary code here.
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

y = df.loc[:,'num']
X = df.iloc[:,:8]

# Select numerical, categorical, and binary columns
numerical_cols = ['age', 'trestbps', 'thalach', 'oldpeak']
categorical_cols = ['cp', 'restecg']
binary_cols = ['sex', 'exang']

# Create a ColumnTransformer with different preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),  # StandardScaler for numerical features
        ('cat', OneHotEncoder(sparse_output = False), categorical_cols),  # OneHotEncoder for categorical features
        ('passthrough', 'passthrough', binary_cols)  # Passthrough for binary features
    ])
transformed_data = preprocessor.fit_transform(X)
transformed_df = pd.DataFrame(transformed_data, columns=preprocessor.get_feature_names_out())
transformed_df.head()


,num__age,num__trestbps,num__thalach,num__oldpeak,cat__cp_1,cat__cp_2,cat__cp_3,cat__cp_4,cat__restecg_0.0,cat__restecg_1.0,cat__restecg_2.0,passthrough__sex,passthrough__exang
0,-2.542347,-0.147076,1.951150,-0.646074,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,-2.414117,-0.716341,0.887744,-0.646074,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,-2.414117,0.422189,1.313106,-0.646074,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,-2.285888,2.129984,1.313106,-0.646074,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,-2.157658,-1.854871,0.462382,-0.646074,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


# **2. Pipeline and Modeling**

- **2.1** Create **three** `Pipeline` objects that take the column transformer from the previous question as the first step and add one or more models as the subsequent steps. You can use any models from `sklearn` or other libraries that are suitable for binary classification. For each pipeline, explain **why** you selected the model(s) and what are their **strengths and weaknesses** for this data set. **(3 Points)**

- **2.2** Use `GridSearchCV` to perform a grid search over the hyperparameters of each pipeline and find the best combination that maximizes the cross-validation score. Report the best parameters and the best score for each pipeline. Then, update the hyperparameters of each pipeline using the best parameters from the grid search. **(6 Points)**

- **2.3** Form a stacking classifier that uses the three pipelines from the previous question as the base estimators and a meta-model as the `final_estimator`. You can choose any model for the meta-model that is suitable for binary classification. Explain **why** you chose the meta-model and how it combines the predictions of the base estimators. Then, use `StratifiedKFold` to perform a cross-validation on the stacking classifier and present the accuracy scores and F1 scores for each fold. Report the mean and the standard deviation of each score in the format of `mean ± std`. For example, `0.85 ± 0.05`. Interpret the results and compare them with the baseline scores from the previous assignment. **(5 Points)**

- **2.4**: Interpret the final results of the stacking classifier and compare its performance with the individual models. Explain how stacking classifier has improved or deteriorated the prediction accuracy and F1 score, and what are the possible reasons for that. **(3 Points)**

<font color='Green'><b>Answer:</b></font>

- **2.1** .....................

**Logistic Regression:**

**Strengths:**

Simple and interpretable.

Works well with linearly separable data.

**Weaknesses:**

Assumes a linear relationship between features and target.

May not perform well if the decision boundary is highly non-linear.

**Random Forest Classifier:**

**Strengths:**

Can handle both numerical and categorical features without much preprocessing.

Can capture non-linear relationships between features and target.

**Weaknesses:**

Can be slow to train on large datasets.

Less interpretable compared to simpler models like logistic regression.

May not perform well with high-dimensional sparse data.

**Support Vector Classifier (SVC):**

**Strengths:**

Effective in high-dimensional spaces, even with small datasets.

Versatile as it can use different kernel functions to capture complex relationships.

**Weaknesses:**

Can be memory-intensive and slow to train on large datasets.

Sensitive to the choice of kernel and its parameters.


In [38]:
# 2.1
# Add necessary code here.
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Pipeline with Random Forest Classifier

pipeline_rf = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])

# Pipeline with Logistic Regression
pipeline_lr = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

# Pipeline with Support Vector Classifier
pipeline_svm = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SVC())
])

<font color='Green'><b>Answer:</b></font>

- **2.2** .....................

In [42]:
# 2.2
# Add necessary code here.
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import make_scorer, accuracy_score, f1_score

# Define parameter grids for Random Forest classifier
param_grid_rf = {
    'classifier__n_estimators': [100, 200, 300],  # Step 1: Number of trees in the forest [100, 200, 300]
    'classifier__max_depth': [5, 10, 15]          # Step 2: Maximum depth of the tree  [5, 10, 15]
}

# Define parameter grids for Logistic Regression classifier
  # Step 1: C: Inverse of regularization strength; smaller values specify stronger regularization; Use: [0.1, 1, 10]
  # Step 2: penalty: Type of penalty; Use: 'l1' for Lasso
  # Step 3: solver: Algorithm to use in the optimization problem; Us: ['liblinear', 'saga']
param_grid_lr = {
    'classifier__C': [0.1, 1, 10],
    'classifier__penalty': ['l1'],
    'classifier__solver': ['liblinear', 'saga']
}


# Define parameter grids for SVM classifier
  # Step 1: C: Regularization parameter; Use: [0.1, 1, 10]
  # Step 2: kernal: Kernel type to be used in the algorithm; Use: ['linear', 'rbf']
param_grid_svm = {
    'classifier__C' :[0.1, 1, 10],
    'classifier__kernel' : ['linear', 'rbf']
}



# Create GridSearchCV instance for Random Forest algorithm
# Parameters:
# Random Forest pipeline, param_grid_rf, 5-fold cross-validation (cv=5), scoring metrics specified in scoring dictionary
# Refit the model based on the F1-score to get the best model
grid_search_rf_with_acc_score = GridSearchCV(pipeline_rf, param_grid_rf, cv=5, scoring='accuracy')
grid_search_rf_with_f1_score = GridSearchCV(pipeline_rf, param_grid_rf, cv=5, scoring='f1')

# Create GridSearchCV instance for Logistic Regression algorithm
# Step 1: Perform grid search using Logistic Regression pipeline
# Step 2: Use parameter combinations defined in param_grid_lr
# Step 3: Use 5-fold cross-validation (cv=5) for model evaluation
# Step 4: Use scoring metrics specified ealier and  n_jobs=-1
# Step 5: Refit the model based on the F1-score to get the best model
grid_search_lr_with_acc_score = GridSearchCV(pipeline_lr, param_grid_lr, cv=5, scoring='accuracy' )
grid_search_lr_with_f1_score = GridSearchCV(pipeline_lr, param_grid_lr, cv=5, scoring='f1' )


# Create GridSearchCV instance for SVM algorithm
# Step 1: Perform grid search using SVM pipeline
# Step 2: Use parameter combinations defined in param_grid_svm
# Step 3: Use 5-fold cross-validation (cv=5) for model evaluation
# Step 4: Use scoring metrics specified ealier and  n_jobs=-1
# Step 5: Refit the model based on the F1-score to get the best model
grid_search_svm_with_acc_score = GridSearchCV(pipeline_svm, param_grid_svm, cv=5, scoring='accuracy')
grid_search_svm_with_f1_score = GridSearchCV(pipeline_svm, param_grid_svm, cv=5, scoring='f1')


#Split X and y into a train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#####################################################################################################################

#Fit the grid search models for the training data


# Fit the Random Forest model
grid_search_rf_with_acc_score.fit(X_train, y_train)
grid_search_rf_with_f1_score.fit(X_train, y_train)

# Fit the Logistic Regression model
grid_search_lr_with_acc_score.fit(X_train, y_train)
grid_search_lr_with_f1_score.fit(X_train, y_train)

# Fit the SVM model
grid_search_svm_with_acc_score.fit(X_train, y_train)
grid_search_svm_with_f1_score.fit(X_train, y_train)

#####################################################################################################################


# Get the best parameters for Random Forest based on F1 score
best_params_rf_with_acc = grid_search_rf_with_acc_score.best_params_
best_params_rf_with_f1 = grid_search_rf_with_f1_score.best_params_

# Get the best parameters for Logistic Regression based on F1 score
best_params_lr_with_acc = grid_search_lr_with_acc_score.best_params_
best_params_lr_with_f1 = grid_search_lr_with_f1_score.best_params_


# Get the best parameters for SVM based on F1 score
best_params_svm_with_acc = grid_search_svm_with_acc_score.best_params_
best_params_svm_with_f1 = grid_search_svm_with_f1_score.best_params_

#####################################################################################################################

print(' ')
print(' ')
print('Best parameters for Random Forest(Based on Accuracy):', best_params_rf_with_acc)
print("Best Accuracy Score on Training Data:", grid_search_rf_with_acc_score.best_score_)
grid_search_rf_with_acc_score.fit(X_test, y_test)#Now we fit for validation data
print("Best Accuracy Score on Validation Data:", grid_search_rf_with_acc_score.best_score_)
print("Best F1 Score on Training Data:", grid_search_rf_with_f1_score.best_score_)
grid_search_rf_with_f1_score.fit(X_test, y_test)#Now we fit for validation data
print("Best F1 Score on Validation Data:", grid_search_rf_with_f1_score.best_score_)
print(' ')
print(' ')
print('Best parameters for Logistic Regression(Based on Accuracy):', best_params_lr_with_acc)
print("Best Accuracy Score on Training Data:", grid_search_lr_with_acc_score.best_score_)
grid_search_lr_with_acc_score.fit(X_test, y_test)#Now we fit for validation data
print("Best Accuracy Score on Validation Data:", grid_search_lr_with_acc_score.best_score_)
print("Best F1 Score on Training Data:", grid_search_lr_with_f1_score.best_score_)
grid_search_lr_with_f1_score.fit(X_test, y_test)#Now we fit for validation data
print("Best F1 Score on Validation Data:", grid_search_lr_with_f1_score.best_score_)
print(' ')
print(' ')
print('Best parameters for SVM(Based on Accuracy):', best_params_svm_with_acc)
print("Best Accuracy Score on Training Data:", grid_search_svm_with_acc_score.best_score_)
grid_search_svm_with_acc_score.fit(X_test, y_test)#Now we fit for validation data
print("Best Accuracy Score on Validation Data:", grid_search_svm_with_acc_score.best_score_)
print("Best F1 Score on Training Data:", grid_search_svm_with_f1_score.best_score_)
grid_search_svm_with_f1_score.fit(X_test, y_test)#Now we fit for validation data
print("Best F1 Score on Validation Data:", grid_search_svm_with_f1_score.best_score_)
print(' ')
print(' ')

#####################################################################################################################

# Rebuild a Random Forest model using the best parameters obtained from the parameter search
#Note best parameters are based on best Accuracy from previous step

rfc = RandomForestClassifier(random_state = 0, max_depth=5, n_estimators=300)
lr = LogisticRegression(random_state = 0, C=1, penalty='l1',solver='liblinear')
svm = SVC(C=0.1, kernel='linear')

# Train the model
rfc.fit(X_train, y_train)
lr.fit(X_train, y_train)
svm.fit(X_train, y_train)
# Evaluate the model on the training and test sets
rfc_train_score = rfc.score(X_train, y_train)
rfc_test_score = rfc.score(X_test, y_test)

lr_train_score = lr.score(X_train, y_train)
lr_test_score = lr.score(X_test, y_test)

svm_train_score = svm.score(X_train, y_train)
svm_test_score = svm.score(X_test, y_test)

print('New Random Forest Classifier Train Score:', rfc_train_score)
print('New Random Forest Classifier Test Score:', rfc_test_score)

print('New Logistic Regression Classifier Train Score:', lr_train_score)
print('New Logistic Regression Classifier Test Score:', lr_test_score)

print('New SVM Train Score:', svm_train_score)
print('New SVM Test Score:', svm_test_score)


/Users/ej/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ej/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ej/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ej/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ej/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ej/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:35

 
 
Best parameters for Random Forest(Based on Accuracy): {'classifier__max_depth': 15, 'classifier__n_estimators': 100}
Best Accuracy Score on Training Data: 0.8212765957446809
Best Accuracy Score on Validation Data: 0.7984848484848486
Best F1 Score on Training Data: 0.7435976488141175
Best F1 Score on Validation Data: 0.7151515151515151
 
 
Best parameters for Random Forest(Based on Accuracy): {'classifier__C': 1, 'classifier__penalty': 'l1', 'classifier__solver': 'liblinear'}
Best Accuracy Score on Training Data: 0.8382978723404255


/Users/ej/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ej/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ej/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ej/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ej/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ej/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:35

Best Accuracy Score on Validation Data: 0.8151515151515151
Best F1 Score on Training Data: 0.7641324056504322


/Users/ej/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ej/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ej/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ej/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ej/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ej/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:35

Best F1 Score on Validation Data: 0.7168253968253968
 
 
Best parameters for Random Forest(Based on Accuracy): {'classifier__C': 0.1, 'classifier__kernel': 'linear'}
Best Accuracy Score on Training Data: 0.8297872340425532
Best Accuracy Score on Validation Data: 0.8136363636363637
Best F1 Score on Training Data: 0.7556944230076659
Best F1 Score on Validation Data: 0.692063492063492
 
 
New Random Forest Classifier Train Score: 0.8978723404255319
New Random Forest Classifier Test Score: 0.7966101694915254
New Logistic Regression Classifier Train Score: 0.8382978723404255
New Logistic Regression Classifier Test Score: 0.8135593220338984
New SVM Train Score: 0.825531914893617
New SVM Test Score: 0.8135593220338984


<font color='Green'><b>Answer:</b></font>

- **2.3** .....................
 Logistic Regression is chosen as the meta-model because it is simple, interpretable, and performs well in many cases of binary classification. It is also less prone to overfitting and works well with stacked models. StackingClassifier combines the predictions of the base estimators by training a meta-model on their outputs. In this case, the meta-model (Logistic Regression) takes the predictions of the base estimators as input features and learns to make the final prediction based on them. StratifiedKFold is used for cross-validation to ensure that each fold has approximately the same proportion of positive and negative samples as the original dataset. This helps in obtaining reliable estimates of the model performance.
 
 In the last, assignment, we used the SVC and DecisionTreeClassifier models to make classification predictions on the wine dataset. We found that we got 70 and 66 percent Training and Testing accuracy(respectively) with the SVC model. We got 97 and 87 percent Training and Testing accuracy(respectively) with the Decision Tree Classifier. From the results of our StackingClassifier, we can see that it does better than the SVC model but slightly worse than the Decision Tree Classifier in terms of Accuracy.

In [37]:
# 2.3
# Add necessary code here.

from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
# Define base estimators
base_estimators = [
    ('rf', pipeline_rf),
    ('lr', pipeline_lr),
    ('svm', pipeline_svm)
]

# Define meta-model
meta_model = LogisticRegression()

# Create StackingClassifier
stacking_classifier = StackingClassifier(estimators=base_estimators, final_estimator=meta_model)

# Perform cross-validation with StratifiedKFold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation then get accuracy and F1 scores for each fold
accuracy_scores = cross_val_score(stacking_classifier, X, y, cv=cv, scoring='accuracy')
f1_scores = cross_val_score(stacking_classifier, X, y, cv=cv, scoring='f1')

# Report mean and standard deviation of accuracy and F1 scores
mean_accuracy = accuracy_scores.mean()
std_accuracy = accuracy_scores.std()
mean_f1 = f1_scores.mean()
std_f1 = f1_scores.std()

print("Accuracy: {:.2f} ± {:.2f}".format(mean_accuracy, std_accuracy))
print("F1 Score: {:.2f} ± {:.2f}".format(mean_f1, std_f1))

Accuracy: 0.81 ± 0.05
F1 Score: 0.71 ± 0.10


<font color='Green'><b>Answer:</b></font>

- **2.4** .....................
The accuracy of StackingClassifier is 0.81 ± 0.05. The F1-Score is 0.71 ± 0.10. The training accuracy and f1 of the RandomForestClassifier is 0.8212765957446809 and 0.7435976488141 respectively. The training accuracy and f1 of the LogisticRegressionModel is 0.8382978723404255 and 0.7641324056504 respectively. The training accuracy and f1 of the SVC Model is 0.8297872340425532 and 0.755694423007 respectively. The accuracy and f1-score of the stacking classifier is in line with the training accuracies of the individual models. This suggests that the stacking classifier has not significantly improved or deteriorated the prediction accuracy compared to the individual models. One reason for this might be that if the individual models have similar biases or make similar errors, the stacking classifier may not provide significant improvements in accuracy. Additionally, if the base models are highly correlated or have similar weaknesses, the stacking classifier's performance may not surpass that of the individual models.



**Bonus Question**: The stacking classifier has achieved a high accuracy and F1 score, but there may be still room for improvement. Suggest **two** possible ways to improve the modeling using the stacking classifier, and explain **how** and **why** they could improve the performance. **(2 points)**

<font color='Green'><b>Answer:</b></font>

Model Diversity:

 Increase the diversity of the base models used in the stacking classifier. Including models with different algorithms, such as decision trees, k-nearest neighbors, or gradient boosting machines, can capture different aspects of the data and reduce the likelihood of correlated errors. Diversity in base models helps the stacking classifier generalize better to unseen data.

Hyperparameter Tuning:

 Explanation: Optimize the hyperparameters of both the base models and the meta-model in the stacking classifier. Fine-tuning hyperparameters can significantly impact model performance by finding the optimal configuration for each model and the ensemble as a whole. By tuning hyperparameters, you can potentially improve the generalization ability of the stacking classifier and achieve better performance on unseen data.